In [3]:
from abc import ABCMeta, abstractmethod
from urllib.parse import urlencode
import requests

In [41]:
class NCBI_Searcher(metaclass=ABCMeta):
    """ 'Interface' que define a utilização da API das databases da NCBI.
    """
    
    search_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    meta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'
        
    def search(self, queryterms: list=None, 
               retmode: str='json', retmax: int=20, retstart: int=0):
        """
        Realiza uma pesquisa NCBI.
        
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param retmode: A forma de retorno. None será em XML.
        @@param retmax: A quantidade de artigos por pesquisa
        @param retstart: O 'offset' de início de retorno. Se setado
            para zero, irá trazer os resultados a partir do primeiro.
            Se setado para 10, irá trazer os resultados a partir do
            décimo.
            
        @return: uma lista de títulos e IDs no formato [(title, id)]
        """
        payload = {"term":self.__search_term(queryterms), "db":self._db, "retmode":retmode,
                   "retmax": retmax, "retstart":retstart}
        url = "%s?%s" % (self.search_url, urlencode(payload))
        
        id_list = requests.get(url).json()['esearchresult']['idlist']
        
        result = self.get_article_metadata(*id_list)['result']
        
        return [(result[uid]['title'], uid) for uid in result['uids']]
    
    def get_article_metadata(self, *args, retmode: str='json'):
        """
        Retorna os metadados do(s) artigo(s).
        
        @param id: ID do artigo. O ID do artigo depende da base de dados.
            O ID de um artigo pode ser diferente entre a PMC e a PubMed
            por exemplo. Podem ser passados vários IDs ao mesmo tempo.
        @param db: base de dados a ser pesquisada. Se for mais de uma,
            pode ser separado por vírgula. As mais úteis serão:
            pmc, pubmed
        @param retmode: A forma de retorno. None será em XML.
        
        @return: o json cru retornado pela API.
        """
        id_list = ','.join([str(x) for x in args])
            
        payload = {"id":id_list, "db":self._db, "retmode":retmode}
        url = "%s?%s" % (self.meta_url, urlencode(payload))
        
        r = requests.get(url).json()
        
        return r
    
    def __search_term(self, queryterms: list, search_type: str=None):
        """Monta o termo de pesquisa completo para mandar para a API."""
        
        if search_type in ['querytext', None]:
            # Retorna simplesmente a busca concatenando com os OR's e AND's
            return "(%s)" % " AND ".join(["(%s)" % " OR ".join(orses)for orses in queryterms])
        
        # Retorna concacentando com os OR'S e AND's, mas embutindo também os campos de pesquisa em cada termo
        return "(%s)" % " AND ".join(["(%s)" % " OR ".join(self.__embutir_fields(orses))for orses in queryterms])
        
        if search_type in [None, 'querytext']:
            # Pesquisa normal, incluindo tudo
            return " AND ".join(["(%s)" % " OR ".join(orses)for orses in queryterms])
    
    def __embutir_fields(self, term: str):
        """Faz uma transformação, embutindo fields no termo de pesquisa.
        Isso é para poder realizar a pesquisa em apenas alguns campos ao invés de todos.
        
        Exemplo: sendo self.__fields = ['title', 'abstract'],
        a chamada 
        `self.__embutir_fields("machine learning")`
        Transforma:
            machine learning ---> (machine learning[title] OR machine learning[abstract])
        """
        
        return "(%s)" % " OR ".join(["%s[%s]" % (term, field) for field in self._fields])
    
    @property
    @abstractmethod
    def _fields(self):
        """Cada subclasse deverá definir quais serão os campos de pesquisa de cada termo.
        O retorno deverá ser uma lista de fields.
        Exemplo: 
        return ['title', 'abstract']
        """
        pass
    
    @property
    @abstractmethod
    def _db(self):
        """Cada subclasse deverá definir o seu banco.
        Exemplo: 
        return 'pmc'
        """
        pass

In [42]:
class PMC_Searcher(NCBI_Searcher):
    """Realiza pesquisas na base PMC."""
    
    @property
    def _fields(self):
        return ['Abstract', 'Body - Key Terms', 'MeSH Terms']
    
    @property
    def _db(self):
        return 'pmc'

In [43]:
technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

queryterms = [technology_queryterms, health_queryterms]

In [46]:
r = PMC_Searcher().search(queryterms=queryterms)
[x[0][:80] for x in r]

['Heterogeneity of Cancer Stem Cells: Rationale for Targeting the Stem Cell Niche',
 'Infections up to 76 days after stroke increase disability and death',
 'Novel Approaches to the Diagnosis of Chronic Disorders of Consciousness: Detecti',
 'Safety and tolerability of spermidine supplementation in mice and older adults w',
 'The Role of Stress-Induced O-GlcNAc Protein Modification in the Regulation of Me',
 'A Decade of Genetic and Metabolomic Contributions to Type 2 Diabetes Risk Predic',
 'The Immune and Non-Immune Pathways That Drive Chronic Gastrointestinal Helminth ',
 'Treating Depression with Transcutaneous Auricular Vagus Nerve Stimulation: State',
 'Adherence to Mediterranean and low-fat diets among heart and lung transplant rec',
 'Proteomic identification and characterization of hepatic glyoxalase 1 dysregulat',
 'Oasis 2: improved online analysis of small RNA-seq data',
 'Designing a behavioral intervention using the COM-B model and the theoretical do',
 'Impaired β-arrest

In [28]:
print('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=machine%20learning%20AND%20acute%20coronary%20syndrome%20AND%20help&tool=my_tool&email=my_email@example.com&retmode=json')

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=machine%20learning%20AND%20acute%20coronary%20syndrome%20AND%20help&tool=my_tool&email=my_email@example.com&retmode=json
